## Google Drive Authentication

In [15]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
# drive = GoogleDrive(gauth)

# from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    print("** Authentication Complete! **")

except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=580813343747-40ut7bhct6qh73nbbvfcpd94l491piv7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61451%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=XDskjI3WnjOOelodJ0BngqZ4HZJSZo&access_type=offline
** Authentication Complete! **


## Google Drive API helper functions

In [22]:
import io
from googleapiclient.http import MediaIoBaseDownload

def getFolderId(service, folderName: str):
    query = "name contains '%s' and mimeType = '%s'" % (folderName, 'application/vnd.google-apps.folder')

    fid = None

    if folderName.startswith('+'):
        return (folderName[1:])

    result = service.files().list(q=query,
                                  pageSize=10, pageToken='',
                                  fields="nextPageToken,files(parents,id,name,mimeType)").execute()
  
    if len(result['files']) == 0:
        print("Folder NOT found")
    else:
        folder = result.get('files')[0]
        fid = folder['id']

    return (fid)

    
def downloadFolder(service, fileId, destinationFolder):
    if not os.path.isdir(destinationFolder):
        os.mkdir(path=destinationFolder)

    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(fileId),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])

    for item in items:
        itemName = item['name']
        itemId = item['id']
        itemType = item['mimeType']
        filePath = destinationFolder + "/" + itemName

        if itemType == 'application/vnd.google-apps.folder':
            downloadFolder(service, itemId, filePath) # Recursive call
            print("Downloaded folder: {0}".format(itemName))
        elif not itemType.startswith('application/'):
            downloadFile(service, itemId, filePath)
        else:
            print("Unsupported file: {0}".format(itemName))


def downloadFile(service, fileId, filePath):
    # Note: The parent folders in filePath must exist
    request = service.files().get_media(fileId=fileId)
    fh = io.FileIO(filePath, mode='wb')
    
    try:
        downloader = MediaIoBaseDownload(fh, request, chunksize=1024*1024)

        done = False
        while done is False:
            status, done = downloader.next_chunk(num_retries = 2)
    finally:
        fh.close()

## Download Trait Files!

In [23]:
import os, shutil

base_filepath = 'Traits'

# Delete previously downloaded trait files
if os.path.isdir(base_filepath):
    shutil.rmtree(base_filepath)

traits_folder_id = getFolderId(service, base_filepath)

downloadFolder(service, traits_folder_id, base_filepath)

Downloaded folder: Patches
Downloaded folder: Baseball cap
Downloaded folder: Flag Patches
Downloaded folder: Patches
Downloaded folder: Beanies
Downloaded folder: Bucket hats
Downloaded folder: Hats
Downloaded folder: Undergarment upper
Downloaded folder: Undergarment lower
Downloaded folder: Sneakers
Downloaded folder: Shoes
Downloaded folder: Short pants
Downloaded folder: Hoodies
Downloaded folder: Long sleeves
Downloaded folder: Surgical masks
Downloaded folder: Jeans
Downloaded folder: Long pants
Downloaded folder: Eaten
Downloaded folder: Crying
Downloaded folder: Angry
Downloaded folder: Neutral
Downloaded folder: Original Yellow bodies
Downloaded folder: Body + Face


## Display Downloaded Traits [TODO]

In [24]:
import ipyplot

rootdir = 'Traits'
labels = []
images = []

for trait_folder in os.listdir(rootdir):
    tf = os.path.join(rootdir, trait_folder)
    
    trait_images = []
    for trait_image in os.listdir(tf):
        ti = os.path.join(rootdir, trait_folder, trait_image)
        images.append(ti)
        labels.append(trait_folder)
    
ipyplot.plot_class_tabs(images, labels, max_imgs_per_tab=10, img_width=300)

NotADirectoryError: [Errno 20] Not a directory: 'Traits/.DS_Store'

## Set trait conditions for layering

In [ ]:
ordered_traits = [
 ['Body + Face'],
# Undergarments for lower
# Undergarments for upper
# Shoes
# Short pants, Long pants
# Long sleeves
# Surgical Masks
# ['Beanie and flags', 'Bucket hats', 'Baseball Cap + Patches']
 ['Beanie and flags', 'Bucket hats', 'Baseball Cap + Patches'],
 ['Surgical masks'],
 ['Undergarments', 'Long pants', 'Short pants'],
 ['Long sleeves'],
 ['Shoes']
]

background_colors = []

assert len(list(set(labels))) == len([item for sublist in ordered_traits for item in sublist])

images_count = 50

In [ ]:
import random
from PIL import Image

label_images_dict = {}
for (image, label) in zip(images, labels):
    if label not in label_images_dict:
        label_images_dict[label] = set()
    label_images_dict[label].add(image)
    
base_file_path = 'Babies/'

# Delete previously created baby files
if os.path.isdir(base_file_path):
    shutil.rmtree(base_file_path)

os.mkdir(base_file_path)
        
        
random_baby_image_file = list(label_images_dict['Body + Face'])[0]
random_baby_image = Image.open(random_baby_image_file)
x, y = random_baby_image.size

for i in range(images_count):
    final_baby_image = Image.new('RGB', (x, y), (228, 150, 150))

    for trait_type in ordered_traits:
        chosen_trait = random.choice(trait_type)
        chosen_image_file = random.choice(list(label_images_dict[chosen_trait]))

        chosen_image = Image.open(chosen_image_file)
        final_baby_image.paste(chosen_image, (0, 0), chosen_image)

    final_baby_image.save(base_file_path + '{:d}_ugly_baby.PNG'.format(i))

print("** Image generation complete! **")

In [ ]:
# Initialize GoogleDriveFile instance with file id.
file1 = drive.CreateFile({'id': <file-id>})

file1.Trash()  # Move file to trash.
file1.UnTrash()  # Move file out of trash.
file1.Delete()  # Permanently delete the file.


img = Image.open('upper_apparel/white_vest.jpg')
img = img.convert("RGBA")

pixdata = img.load()

width, height = img.size
for y in range(height):
    for x in range(width):
        if pixdata[x, y] == (255, 255, 255, 255):
            pixdata[x, y] = (255, 255, 255, 0)
            
img.save("processed_white_vest.png", "PNG")